In [3]:
from dotenv import load_dotenv
import os

from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

In [4]:
load_dotenv(dotenv_path='.env')
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# Few Shot Prompt - provide a few examples in the template
examples = [
    {
        "texto": "O time fez uma excelente apresentação e marcou vários gols",
        "resposta": "sentimento: positivo, tema: time"
    },
    {
        "texto": "O restaurante tem um serviço ruim e a comida deixa a desejar",
        "resposta": "sentimento: negativo, tema: restaurante"
    },
    {
        "texto": "O podcast tem uma participação razoável mas sem grandes destaques.",
        "resposta": "sentimento: neutro, tema: podcast"
    },
]


In [6]:
examples

[{'texto': 'O time fez uma excelente apresentação e marcou vários gols',
  'resposta': 'sentimento: positivo, tema: time'},
 {'texto': 'O restaurante tem um serviço ruim e a comida deixa a desejar',
  'resposta': 'sentimento: negativo, tema: restaurante'},
 {'texto': 'O podcast tem uma participação razoável mas sem grandes destaques.',
  'resposta': 'sentimento: neutro, tema: podcast'}]

In [7]:
examples.append({'texto': 'O restaurante era bom a música agradável.',
   'resposta': 'sentimento: positivo, tema: restaurante'})


In [8]:
examples

[{'texto': 'O time fez uma excelente apresentação e marcou vários gols',
  'resposta': 'sentimento: positivo, tema: time'},
 {'texto': 'O restaurante tem um serviço ruim e a comida deixa a desejar',
  'resposta': 'sentimento: negativo, tema: restaurante'},
 {'texto': 'O podcast tem uma participação razoável mas sem grandes destaques.',
  'resposta': 'sentimento: neutro, tema: podcast'},
 {'texto': 'O restaurante era bom a música agradável.',
  'resposta': 'sentimento: positivo, tema: restaurante'}]

In [9]:
example_format_template = """
Texto: {texto}
Resposta: {resposta}
"""

example_prompt = PromptTemplate(
    input_variable=["texto","resposta"],
    template = example_format_template
)

In [16]:
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix ="Classifique o sentimento e o tema do texto a seguir:",
    suffix = "Texto: {input} em formato JSON",
    input_variables=["input"]
)

In [17]:
input_text = "O filme foi interessante, mas a atuação deixou a desejar."
prompt_output = prompt.format(input=input_text)
#print(prompt_output)

In [18]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
response = chain.invoke({"input":input_text})
print(response)

```json
{
  "sentimento": "neutro",
  "tema": "filme"
}
```


In [20]:
json_data = response.strip()
json_data = json_data.replace("```json\n","")
json_data = json_data.replace("\n```","")
print(json_data)

{
  "sentimento": "neutro",
  "tema": "filme"
}


In [90]:
import json
try:
    dados = json.loads(json_data)
    print("sentimento: ", dados.get("sentimento"))
    print("tema: ", dados.get("tema"))
except Exception as e :
    print(f"JSON não identificado {e}")

sentimento:  neutro
tema:  filme
